In [1]:
import pandas as pd
from bs4 import BeautifulSoup
from dateutil.relativedelta import relativedelta
import requests


import warnings;
warnings.simplefilter('ignore')

In [ ]:
class ExternalData:
    naming_report_files = {
            "covid": "$country$_CUSTOMER_CATALOG_$year$_$month$.csv",
            "temperature": "$country$_PRODUCT_CATALOG_$year$_$month$.csv",
            "inflation": "$country$_PROMO_HEADER_$year$$month$.csv",
    }

    country_name_mapping = {
            "argentina": "AR",
            "uruguay": "UY",
            "guatemala": "GT",  
            "costarica": "CR",
            "nicaragua": "NI",
            "panama": "PA",
            "colombia": "CO",
            "mexico":"MX"
    }
    
    def __init__(self, fecha:str, country:str):
        self.country = country
        self.fecha = pd.to_datetime(fecha, format="%Y%m")
        self.fecha_inicial = self.fecha.replace(day=1).strftime("%Y%m%d")
        self.fecha_final = self.fecha.replace(day=self.fecha.days_in_month).strftime("%Y%m%d")
        self.df_coordenadas = pd.DataFrame([
                            ['argentina', 'ARGENTINA',-34.6579,-58.4392],
                            ['costarica', 'COSTA RICA',9.9417, -84.1184],
                            ['guatemala', 'GUATEMALA',14.6243,-90.5187],
                            ['nicaragua', 'NICARAGUA',12.14,-86.26],
                            ['panama', 'PANAMA',9.0277,-79.4876],
                            ['uruguay', 'URUGUAY',-34.89,-56.18],
                            ['colombia','Cali',3.434, -76.5268],
                            ['colombia','Girardot',4.2968, -74.8053],
                            ['colombia','Villavicencio',4.1331, -73.6226],
                            ['colombia','Valledupar',10.4658, -73.2613],
                            ['colombia','Medellin',6.2769, -75.5977],
                            ['colombia','Bogota Norte',4.690471130761603, -74.05623579201409],
                            ['colombia','Ibague',4.423846317899223, -75.18598098742774],
                            ['colombia','Neiva',2.954316738716088, -75.30002584784135],
                            ['colombia','Cartagena',10.422270311506267, -75.54526131436982],
                            ['colombia','Santa Marta',11.19099019233421, -74.2089124080734],
                            ['colombia','Corozal',9.320342390753085, -75.30488201896188],
                            ['colombia','Bogota Sur',4.60247457965247, -74.16628609526084],
                            ['colombia','Cucuta',7.859971764120941, -72.51305693246773],
                            ['colombia','Bucaramanga',7.09155843319789, -73.12502630363817],
                            ['colombia','Barranquilla',10.956201902276364, -74.78604599506897],
                            ['colombia','Barrancabermeja',7.076652411989663, -73.85590387295004],
                            ['colombia','Duitama',5.818664750516205, -73.02344318466417],
                            ['colombia','Pereira',4.810800178684645, -75.79420800365315],
                            ['colombia','Yopal',5.33418030700621, -72.38657000555652],
                            ['colombia','Monteria',8.749838823094498, -75.89074854595145],
                            ['colombia','San Andres',10.401856375243826, -75.52176296714006],
                            ['colombia','San Gil',6.567241659584618, -73.1490755090324],
                            ['colombia','Aguachica',8.309398818632083, -73.60134590362769],
                            ['colombia','Tunja',5.508357007510656, -73.35815578656472],
                            ['colombia','Argentina',-34.6579, -58.4392],
                            ])
        
        self.df_coordenadas.columns = ['Country','Region','Latitude', 'Longitude']
    
    def temperature(self)-> pd.DataFrame:
        df_temperature_country = self.df_coordenadas[(self.df_coordenadas['Country'] == self.country)]
        df_temperature_final = pd.DataFrame()
        for reg in df_temperature_country['Region'].unique():
            df_temperature_alpha = df_temperature_country[(df_temperature_country['Region'] == reg)]
            latitude_temp = df_temperature_alpha.Latitude.iloc[0]
            longitude_temp = df_temperature_alpha.Longitude.iloc[0]

            url_temperature = f'https://power.larc.nasa.gov/api/temporal/daily/point?parameters=T2M,T2MDEW,T2MWET,TS,T2M_RANGE,T2M_MAX,T2M_MIN,QV2M,RH2M,PRECTOTCORR&community=RE&longitude={longitude_temp}&latitude={latitude_temp}&start={self.fecha_inicial}&end={self.fecha_final}&format=CSV'
            df_temperature = pd.read_csv(url_temperature, sep=',', skiprows=18)
            df_temperature_final = pd.concat([df_temperature_final, df_temperature])

        return df_temperature_final
    
    def covid(self)-> pd.DataFrame:
        if self.country == 'guatemala':
            f_initial_sep = pd.to_datetime(self.fecha_inicial).strftime("%Y-%m-%d")
            f_final_sep = pd.to_datetime(self.fecha_final).strftime("%Y-%m-%d")
        else:
            f_initial_sep = pd.to_datetime(self.fecha_inicial).strftime("%d-%m-%Y")
            f_final_sep = pd.to_datetime(self.fecha_final).strftime("%d-%m-%Y")

        df_covid = []
        for day in pd.date_range(f_initial_sep, f_final_sep):
            if self.country == 'argentina':
                df_covid.append(['AR','Argentina',None,None,None,None,None,None,day,18,44,-7,6,46,4])
            elif self.country == 'costarica':
                df_covid.append(['CR','Costa Rica',None,None,None,None,None,None,day,-1,22,-10,-12,15,8])
            elif self.country == 'guatemala':
                df_covid.append(['GT','Guatemala',None,None,None,None,None,None,day,36,43,22,4,29,10])
            elif self.country == 'nicaragua':
                df_covid.append(['NI','Nicaragua',None,None,None,None,None,None,day,40,60,35,24,27,8])
            elif self.country == 'panama':
                df_covid.append(['PA','Panama',None,None,None,None,None,None,day,1,23,-25,8,13,9])
            elif self.country == 'uruguay':
                df_covid.append(['UY','Uruguay',None,None,None,None,None,None,day,-4,20,-22,-11,24,4])

        df_covid = pd.DataFrame(df_covid)
        df_covid.columns = ['COUNTRY_REGION_CODE','COUNTRY_REGION','SUB_REGION_1','SUB_REGION_2','METRO_AREA','ISO_3166_2_CODE','CENSUS_FIPS_CODE','PLACE_ID','DATE','RETAIL_AND_RECREATION_PERCENT_CHANGE_FROM_BASELINE','GROCERY_AND_PHARMACY_PERCENT_CHANGE_FROM_BASELINE','PARKS_PERCENT_CHANGE_FROM_BASELINE','TRANSIT_STATIONS_PERCENT_CHANGE_FROM_BASELINE','WORKPLACES_PERCENT_CHANGE_FROM_BASELINE','RESIDENTIAL_PERCENT_CHANGE_FROM_BASELINE']
        
        return df_covid
    
    def inflation(self)-> pd.DataFrame:
        week_list = pd.read_csv("CALENDAR.csv")
        week_list['DAY'] = pd.to_datetime(week_list['DAY'])
        week_list['DIA'] = week_list['DAY'].dt.day_name()
        week_list = week_list[(week_list['DIA'] == 'Sunday')]
        week_list['FECHA'] = week_list['YEAR'].astype(str) + week_list['MONTH'].apply(lambda x: f'{x:02d}')
        week_list = week_list[['FECHA','WEEK']]
        week_list = week_list[(week_list['FECHA'] == self.fecha.strftime("%Y%m"))]['WEEK'].tolist()
        df_inflation = pd.DataFrame()

        req_headers = {
            'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
            'accept-encoding': 'gzip, deflate, br',
            'accept-language': 'en-US,en;q=0.8',
            'upgrade-insecure-requests': '1',
            'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'
            }
        
        if self.country in ['costarica','panama','guatemala','nicaragua']:
            lista_fechas = [(self.fecha - relativedelta(months = i )).strftime("%Y%#m") for i in range(1, -1, -1)]
            lista_fechas = "-".join(lista_fechas)
            
            url_inflation_ca = f"https://www.secmca.org/chart/?parent=Precios&scid=0&cid=0&scsid=0&son=%C3%8Dndice%20de%20precios%20al%20consumidor&url=1/14/3-50-4-161/39/265/19-258/78-313-314-315/{lista_fechas}-&all_vars=1|IPC%20general*2|IPC%20variaci%C3%B3n%20mensual*3|IPC%20variaci%C3%B3n%20interanual*4|IPC%20variaci%C3%B3n%20acumulada%20con%20diciembre%20anterior"

            r1 = requests.get(url_inflation_ca, headers = req_headers, verify=False)
            r1 = BeautifulSoup(r1.content, 'html')
            r1 = r1.findAll('td')

            alpha = []
            for i in range(len(r1)):
                alpha.append(r1[i].text.strip())
                
            dict_countries = {0:'costarica',
                            1:'guatemala',
                            2:'nicaragua',
                            3:'panama'
                            }

            betha = []
            for n_row in range(int(len(alpha)/17)):
                gamma = alpha[(n_row*17)+1:17*(n_row+1)]
                for n_countries in range(int(len(gamma)/4)):
                    iota = gamma[(n_countries * 4):4 * (n_countries+1)]
                    iota.append(dict_countries[n_countries])
                    betha.append(iota)

            df_betha = pd.DataFrame(betha[4:], columns =  ['IPC general',
                                'IPC variación mensual',
                                'IPC variación interanual',
                                'IPC variación acumulada con diciembre anterior',
                                'Pais'])

            inflacion_col = 'IPC variación interanual' if self.country == 'guatemala' else 'IPC variación mensual'

            for week in week_list:
                epsilon = pd.DataFrame([{
                    'YEAR': self.fecha.strftime("%Y%m")[:4],
                    'WEEK': week,
                    'INFLATION': df_betha[df_betha['Pais'] == self.country][inflacion_col].iloc[0]
                }])
                df_inflation = pd.concat([df_inflation, epsilon], axis=0)
        
        elif self.country == 'argentina':
            fecha_ar = (self.fecha).strftime("%Y%m")
            
            url_inflation = f'https://www.indec.gob.ar/ftp/cuadros/economia/sh_ipc_{fecha_ar[4:]}_{fecha_ar[2:4]}.xls'
            df_inflation_argentina = pd.read_excel(url_inflation, header=None)
            

            for week in week_list:
                epsilon = pd.DataFrame([{
                    'YEAR': self.fecha.strftime("%Y%m")[:4],
                    'WEEK': week,
                    'INFLATION': df_inflation_argentina.loc[df_inflation_argentina[0] == 'Nivel general'].iloc[0, -1]
                }])
                df_inflation = pd.concat([df_inflation, epsilon], axis=0)
        
        elif self.country == 'uruguay':
            pass
        elif self.country == 'colombia':
            pass
            
        
        return df_inflation

In [114]:
import requests
import pandas as pd
from io import BytesIO

month_short = 'feb'
url_excel = f'https://www.dane.gov.co/files/operaciones/IPC/{month_short}2025/anex-IPC-CiudadesMensuales-{month_short}2025.xlsx'
df_inflation_colombia_mensual = pd.read_excel(url_excel, header=None)
df_inflation_colombia_mensual = df_inflation_colombia_mensual.loc[df_inflation_colombia_mensual[(df_inflation_colombia_mensual[0] == ' Variaciones mensuales 2025')].index[0] + 1:]
df_inflation_colombia_mensual = df_inflation_colombia_mensual.loc[:df_inflation_colombia_mensual[(df_inflation_colombia_mensual[0] == 'Otras áreas urbanas')].index[0] -1]
df_inflation_colombia_mensual.columns = df_inflation_colombia_mensual.iloc[0]
df_inflation_colombia_mensual = df_inflation_colombia_mensual[2:]
df_inflation_colombia_mensual = df_inflation_colombia_mensual.rename(columns={'Mes':'Distribuidora'})
month_long = [m for m in df_inflation_colombia_mensual.columns if month_short in m[:3].lower()]
col_select =  ['Distribuidora'] + month_long + ['Año corrido']
df_inflation_colombia_mensual = df_inflation_colombia_mensual[col_select]
df_inflation_colombia_mensual = df_inflation_colombia_mensual.rename(columns={month_long[0]:'Variacion_Mensual_PCT',
                                                                              'Año corrido':'Variacion_YTD_PCT'})
df_inflation_colombia_mensual



226,Distribuidora,Variacion_Mensual_PCT,Variacion_YTD_PCT
228,Medellín,0.62,1.68
229,Barranquilla,1.29,2.23
230,Bogotá D.C.,1.53,2.43
231,Cartagena,0.92,1.88
232,Tunja,1.14,2.23
233,Manizales,1.15,2.16
234,Florencia,0.95,1.63
235,Popayán,1.45,2.37
236,Valledupar,0.87,1.32
237,Montería,0.83,1.34


'Febrero'

226,Distribuidora,Febrero,Año corrido
228,Medellín,0.62,1.68
229,Barranquilla,1.29,2.23
230,Bogotá D.C.,1.53,2.43
231,Cartagena,0.92,1.88
232,Tunja,1.14,2.23
233,Manizales,1.15,2.16
234,Florencia,0.95,1.63
235,Popayán,1.45,2.37
236,Valledupar,0.87,1.32
237,Montería,0.83,1.34


In [108]:
import requests
import pandas as pd
from io import BytesIO

month_short = 'feb'
url_excel = f'https://www.dane.gov.co/files/operaciones/IPC/{month_short}2025/anex-IPC-CiudadesAnuales-{month_short}2025.xlsx'
df_inflation_colombia_anual = pd.read_excel(url_excel, header=None)
df_inflation_colombia_anual = df_inflation_colombia_anual.loc[df_inflation_colombia_anual[(df_inflation_colombia_anual[1] == 'Ciudad')].index[0]:]
df_inflation_colombia_anual = df_inflation_colombia_anual.loc[:df_inflation_colombia_anual[(df_inflation_colombia_anual[1] == 'Otras áreas urbanas')].index[0] -1]
df_inflation_colombia_anual.columns = df_inflation_colombia_anual.iloc[0]
df_inflation_colombia_anual = df_inflation_colombia_anual[1:]
df_inflation_colombia_anual = df_inflation_colombia_anual.rename(columns={'Ciudad':'Distribuidora'})
month_long = [m for m in df_inflation_colombia_anual.columns if month_short in m[:3].lower()]
col_select =  ['Distribuidora'] + month_long
df_inflation_colombia_anual = df_inflation_colombia_anual[col_select]
df_inflation_colombia_anual


7,Distribuidora,Febrero
8,Medellín,4.96
9,Barranquilla,3.94
10,Bogotá D.C.,6.07
11,Cartagena,4.84
12,Tunja,3.81
13,Manizales,4.65
14,Florencia,4.97
15,Popayán,5.38
16,Valledupar,3.89
17,Monteria,4.31


In [ ]:
df_inflation_colombia

202502,Medellin,149.84,4.96,1.68,0.62

Periodo,Distribuidora,IPC,Variacion_Anual_PCT,Variacion_YTD_PCT,Variacion_Mensual_PCT



226,Distribuidora,Enero,Febrero,Marzo,Abril,Mayo,Junio,Julio,Agosto,Septiembre,Octubre,Noviembre,Diciembre,Año corrido
228,Medellín,1.05,0.62,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.68
229,Barranquilla,0.93,1.29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.23
230,Bogotá D.C.,0.89,1.53,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.43
231,Cartagena,0.95,0.92,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.88
232,Tunja,1.08,1.14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.23
233,Manizales,1,1.15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.16
234,Florencia,0.67,0.95,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.63
235,Popayán,0.9,1.45,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.37
236,Valledupar,0.44,0.87,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.32
237,Montería,0.51,0.83,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.34


In [59]:
df_inflation_colombia[df_inflation_colombia[(df_inflation_colombia[0] == ' Variaciones mensuales 2025')].index[0]: df_inflation_colombia[(df_inflation_colombia[0] == ' Otras áreas urbanas')].index[0]	]


IndexError: index 0 is out of bounds for axis 0 with size 0

In [43]:
from datetime import datetime

list_columns_month = ['Mes','Enero','Febrero','Marzo','Abril','Mayo','Junio','Julio','Agosto','Septiembre','Octubre','Noviembre','Diciembre']
list_columns_month_1 = list_columns_month[1:]
# Obtener la fecha y hora actual
fecha_actual = datetime.now()

# Obtener el año actual
año_actual = fecha_actual.year

# Obtener el mes actual en letras
mes_actual = fecha_actual.month-1
mes_anterior = fecha_actual.month-2
nombre_mes_anterior = list_columns_month_1[mes_anterior]
nombre_mes_actual = list_columns_month_1[mes_actual]
ruta_actual = nombre_mes_actual[:3].lower()+str(año_actual)
ruta_anterior = nombre_mes_anterior[:3].lower()+str([año_actual-1 if mes_actual == 0 else año_actual][0])

# try:
url_ipc = f'https://www.dane.gov.co/files/operaciones/IPC/anex-IPC-{ruta_actual}.xlsx'
url_ipc_last = f'https://www.dane.gov.co/files/operaciones/IPC/anex-IPC-{ruta_anterior}.xlsx'
#     if requests.get(url_ipc).status_code == 200:
#         df_ipc_temp = pd.read_excel(url_ipc)
#         print(url_ipc)
#     else: 
#         if requests.get(url_ipc_last).status_code == 200:
#             df_ipc_temp = pd.read_excel(url_ipc_last)
#             print(url_ipc_last)
#         else: 
#             print(url_ipc_last)
#             raise RuntimeError(f'Failed to connect {url_ipc_last}')
# except Exception as inst:
#     print(type(inst))    # the exception type

# df_ipc_temp = df_ipc_temp[df_ipc_temp['Unnamed: 0'].isin(list_columns_month)]
# df_ipc_temp.columns = [str(x).replace(".0","") for x in list(df_ipc_temp.iloc[0])]
# df_ipc_temp = df_ipc_temp[1:].set_index('Mes')

# ipc_temp = df_ipc_temp.copy()
# for year in list(ipc_temp.columns[1:]):
#     ipc_temp[year] = (df_ipc_temp[year]/ df_ipc_temp[str(int(year)-1)]) -1
# ipc_temp = ipc_temp[list(ipc_temp.columns[1:])]

# ipc = pd.DataFrame()
# for year in list(ipc_temp.columns):
#     for num, month in enumerate(list(ipc_temp.index)):
#         ipc_itr = pd.DataFrame()
#         ipc_itr['año_mes'] = [datetime(int(year),num+1,1).strftime('%Y-%m')]
#         ipc_itr['ipc'] = [ipc_temp[year][ipc_temp[year].index == month].iloc[0]]
#         ipc = pd.concat([ipc, ipc_itr], axis= 0)
# ipc = ipc[~(ipc['ipc'].isnull())]

In [ ]:
https://www.dane.gov.co/files/operaciones/IPC/feb2025/anex-IPC-CiudadesMensuales-feb2025.xlsx

'https://www.dane.gov.co/files/operaciones/IPC/anex-IPC-abr2025.xlsx'

In [29]:
external_data = ExternalData('202503','argentina')
df_inflation = external_data.inflation()
df_inflation

,YEAR,WEEK,INFLATION
0,2025,9,2.4
0,2025,10,2.4
0,2025,11,2.4
0,2025,12,2.4
0,2025,13,2.4


In [ ]:
external_data = ExternalData('202502','argentina')
df_temperature_final = external_data.temperature()
df_temperature_final

In [ ]:
external_data = ExternalData('202502','argentina')
df_covid_final = external_data.covid()
df_covid_final